<a href="https://colab.research.google.com/github/PromitHal/Pytorch/blob/master/Linear_Regression_With_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
x=torch.tensor([1,2,3])

In [7]:
x=x.reshape(-1,3)

In [9]:
y=torch.tensor([[1,2,3],[4,5,6],[7,8,9]])
y

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [11]:
y=y.reshape(1,9)
y

tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [12]:
y=y.squeeze(dim=0)


In [13]:
y

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [15]:
y.shape

torch.Size([9])

In [18]:
#Input(temp,rainfall,humidity)
import numpy as np
inputs=np.array([
                 [73,67,43],
                 [91,88,64],
                 [87,134,58],
                 [102,43,37],
                 [69,96,70]
],dtype='float32')
inputs

array([[ 73.,  67.,  43.],
       [ 91.,  88.,  64.],
       [ 87., 134.,  58.],
       [102.,  43.,  37.],
       [ 69.,  96.,  70.]], dtype=float32)

In [19]:
#Targets (apples,oranges)
targets=np.array([
                  [56,70],
                  [81,101],
                  [119,133],
                  [22,37],
                  [103,119]
],dtype='float32')
targets

array([[ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.]], dtype=float32)

In [20]:
#Convert inputs and targets to tensors
inputs=torch.from_numpy(inputs)
targets=torch.from_numpy(targets)
print(input)
print(targets)

<bound method Kernel.raw_input of <google.colab._kernel.Kernel object at 0x7f42c4a45550>>
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


**Linear Regression from Scratch**

In [22]:
w=torch.randn(2,3,requires_grad=True) # Weights
b=torch.randn(2,requires_grad=True) #Biases
print(w)
print(b)

tensor([[ 0.1571, -0.3283,  1.1905],
        [-1.2369, -0.2996,  1.1666]], requires_grad=True)
tensor([1.3394, 1.5431], requires_grad=True)


In [23]:
def model(x):
  return x @ w.t()+b
  # @ represents matrix multiplication in PyTorch,and the .t method returns the transpose of a tensor
  

In [24]:
#Generate Predictions
preds=model(inputs)
print(preds)

tensor([[ 42.0087, -58.6609],
        [ 62.9445, -62.7186],
        [ 40.0728, -78.5536],
        [ 47.3008, -94.3390],
        [ 64.0045, -30.9046]], grad_fn=<AddBackward0>)


In [25]:
#Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


**Loss Function**

In [27]:
diff=targets-preds
diff_sqr=diff*diff #Element wise multiplication
torch.sum(diff_sqr)/diff.numel()

tensor(13674.5674, grad_fn=<DivBackward0>)

In [28]:
#MSE loss
def mse(t1,t2):
  diff=t1-t2
  return torch.sum(diff*diff)/diff.numel()

In [29]:
#Compute loss
loss=mse(preds,targets)
print(loss)

tensor(13674.5674, grad_fn=<DivBackward0>)


**Compute  gradients**

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t to the weights and biases , because they have requires_grad set to True. The derivative of the loss w.r.t the weights matrix is itself a matrix,with the same dimensions.

In [30]:
#Compute gradients
loss.backward()

In [31]:
#Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.1571, -0.3283,  1.1905],
        [-1.2369, -0.2996,  1.1666]], requires_grad=True)
tensor([[ -1928.2180,  -3151.6348,  -1625.7019],
        [-13287.1602, -14282.8232,  -8726.6758]])


In [32]:
print(b)
print(b.grad)

tensor([1.3394, 1.5431], requires_grad=True)
tensor([ -24.9337, -157.0353])


Before we proceed, we reset the gradients to zero by calling .zero_() method. We need to this,because PyTorch accumulates,gradients i.e. the next time we call .backward on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results


In [34]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


**Adjust weights and biases using gradient descent**

In [35]:
#Generate predictions
preds=model(inputs)
print(preds)

tensor([[ 42.0087, -58.6609],
        [ 62.9445, -62.7186],
        [ 40.0728, -78.5536],
        [ 47.3008, -94.3390],
        [ 64.0045, -30.9046]], grad_fn=<AddBackward0>)


In [38]:
#Calculate the loss
loss=mse(preds,targets)
print(loss)

tensor(13674.5674, grad_fn=<DivBackward0>)


In [39]:
#Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -1928.2180,  -3151.6348,  -1625.7019],
        [-13287.1602, -14282.8232,  -8726.6758]])
tensor([ -24.9337, -157.0353])


In [40]:
#Adjust weights and reset gradients
with torch.no_grad():
  w-=w.grad*1e-5
  b-=b.grad*1e-5
  w.grad.zero_()
  b.grad.zero_()


In [41]:
print(w)
print(b)

tensor([[ 0.1764, -0.2967,  1.2068],
        [-1.1040, -0.1568,  1.2539]], requires_grad=True)
tensor([1.3396, 1.5447], requires_grad=True)


In [42]:
#loss
preds=model(inputs)
loss=mse(preds,targets)
print(loss)

tensor(9367.4551, grad_fn=<DivBackward0>)


**Train  for multiple epochs**

In [43]:
for i in range(100):
  preds=model(inputs)
  loss=mse(preds,targets)
  loss.backward()
  with torch.no_grad():
    w-=w.grad*1e-5
    b-=b.grad*1e-5
    w.grad.zero_()
    b.grad.zero_()
    

In [44]:
#Calculate loss
preds=model(inputs)
loss=mse(preds,targets)
print(preds)
print(loss)

tensor([[ 61.6217,  69.9505],
        [ 90.5097, 105.6870],
        [ 92.6495, 122.1299],
        [ 46.2162,  33.9531],
        [101.9607, 129.9583]], grad_fn=<AddBackward0>)
tensor(167.3391, grad_fn=<DivBackward0>)


In [45]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])